In [ ]:
import numpy as np
from tqdm import tqdm, trange
import pickle
from rdkit import Chem, DataStructs
import seaborn as sns
import pandas as pd
import random
import re


# For plotting
import plotly.io as plt_io
import plotly.graph_objects as go

# Dimensionality reduction and clustering
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import umap
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

from rdkit import Chem
from rdkit.Chem import Draw, rdmolops
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')  # switch off RDKit warning messages

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import Lipinski
from rdkit.Chem.AllChem import CalcNumAtomStereoCenters
from rdkit.Chem.GraphDescriptors import BertzCT
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles
from rdkit.Chem import AllChem, rdmolops
from rdkit.DataStructs import FingerprintSimilarity
from rdkit.Chem import MACCSkeys

# suppress Chem.MolFromSmiles error output
from rdkit import rdBase
import sys
import os
rdBase.DisableLog('rdApp.error')
# import from rdkit.Contrib module
from rdkit.Chem import RDConfig
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer
sys.path.append(os.path.join(RDConfig.RDContribDir, 'NP_Score'))
import npscorer

In [ ]:
exp_smiles = pd.read_csv('./data/Experimental_Ligands.csv')
print('Dataset:', exp_smiles.shape)

In [ ]:
df_generated = pd.read_csv('./results/generative_model/Generated_Practical_Set.csv')
df_generated = df_generated[['smiles']]
df_generated

In [ ]:
exp_smiles_list = list(exp_smiles['smiles'])
print('Number of real molecules :', len(exp_smiles_list))
df_generated_list = list(df_generated['smiles'])
print('Number of generated molecules :', len(df_generated_list))

In [ ]:

# Tanimoto Similarity
mol_real = [Chem.MolFromSmiles(x) for x in exp_smiles_list]
#fp_real = [AllChem.GetMorganFingerprintAsBitVect(y, 2) for y in mol_real]
fp_real = [Chem.RDKFingerprint(y) for y in mol_real]
fp_real = [MACCSkeys.GenMACCSKeys(y) for y in mol_real]

mol_gen = [Chem.MolFromSmiles(x) for x in df_generated_list if x is not None]
#fp_gen = [AllChem.GetMorganFingerprintAsBitVect(y,2) for y in mol_gen]
fp_gen = [Chem.RDKFingerprint(y) for y in mol_gen]
fp_gen = [MACCSkeys.GenMACCSKeys(y) for y in mol_gen]

similarities_ij= []

for i in fp_real:
    for j in fp_gen:
        similarities_ij.append(DataStructs.FingerprintSimilarity(i, j))

similarities_ii= []        
for i in fp_real:
    for j in fp_real:
        similarities_ii.append(DataStructs.FingerprintSimilarity(i, j))
        
similarities_jj= []        
for i in fp_gen:
    for j in fp_gen:
        similarities_jj.append(DataStructs.FingerprintSimilarity(i, j))

similarities_ij = np.asarray(similarities_ij, dtype=np.float32)
similarities_ij = np.sort(similarities_ij)

similarities_ii = np.asarray(similarities_ii, dtype=np.float32)
similarities_ii = np.sort(similarities_ii)

similarities_jj = np.asarray(similarities_jj, dtype=np.float32)
similarities_jj = np.sort(similarities_jj)

# Plotting with shading
plt.figure(figsize=(6, 4))


p = sns.kdeplot(similarities_jj,  fill=True, #color='green',
                label='Generated = ' + str(np.round(np.mean(similarities_jj), 2)), linewidth=3,
                shade=True, alpha=.3)
p = sns.kdeplot(similarities_ii,  fill=True, #color='red',
                label='Experimental = ' + str(np.round(np.mean(similarities_ii), 2)), linewidth=3,
                shade=True, alpha=.3)
p = sns.kdeplot(similarities_ij,  fill=True, #color='blue',
                label='Crossed = ' + str(np.round(np.mean(similarities_ij), 2)), linewidth=3,
                shade=True, alpha=.3)

p.legend(bbox_to_anchor=(1.00, 1.00), loc='best', fontsize=12)
#p.set(xlabel='Tanimoto coefficient', title='Distribution of Tanimoto coefficient with mean value')
#plt.tight_layout()
p.set_xlabel('Tanimoto coefficient', fontsize=14)  # Adjust the font size as needed
p.set_ylabel('Density', fontsize=14)  # Adjust the font size as needed
plt.tight_layout()
plt.savefig('./results/TANIMOTO_experimental_vs_generated.png',dpi = 600)
plt.show()


In [ ]:

# Tanimoto Similarity
mol_real = [Chem.MolFromSmiles(x) for x in exp_smiles_list]
#fp_real = [AllChem.GetMorganFingerprintAsBitVect(y, 2) for y in mol_real]
fp_real = [Chem.RDKFingerprint(y) for y in mol_real]
fp_real = [MACCSkeys.GenMACCSKeys(y) for y in mol_real]

mol_gen = [Chem.MolFromSmiles(x) for x in df_generated_list if x is not None]
#fp_gen = [AllChem.GetMorganFingerprintAsBitVect(y,2) for y in mol_gen]
fp_gen = [Chem.RDKFingerprint(y) for y in mol_gen]
fp_gen = [MACCSkeys.GenMACCSKeys(y) for y in mol_gen]

similarities_ij= []

for i in fp_real:
    for j in fp_gen:
        similarities_ij.append(DataStructs.FingerprintSimilarity(i, j))

similarities_ii= []        
for i in fp_real:
    for j in fp_real:
        similarities_ii.append(DataStructs.FingerprintSimilarity(i, j))
        
similarities_jj= []        
for i in fp_gen:
    for j in fp_gen:
        similarities_jj.append(DataStructs.FingerprintSimilarity(i, j))
        


similarities_ij = np.asarray(similarities_ij, dtype=np.float32)
similarities_ij = np.sort(similarities_ij)

similarities_ii = np.asarray(similarities_ii, dtype=np.float32)
similarities_ii = np.sort(similarities_ii)

similarities_jj = np.asarray(similarities_jj, dtype=np.float32)
similarities_jj = np.sort(similarities_jj)

# Plotting with shading
plt.figure(figsize=(6, 4))


p = sns.kdeplot(similarities_ij,  fill=True, color='green',
                label='Mean Tanimoto index = ' + str(np.round(np.mean(similarities_ij), 2)), linewidth=3,
                shade=True, alpha=.3)

p.legend(bbox_to_anchor=(1.00, 1.00), loc='best', fontsize=12)
#p.set(xlabel='Tanimoto coefficient', title='Distribution of Tanimoto coefficient with mean value')
#plt.tight_layout()
p.set_xlabel('Tanimoto coefficient', fontsize=14)  # Adjust the font size as needed
p.set_ylabel('Density', fontsize=14)  # Adjust the font size as needed
plt.tight_layout()
plt.savefig('./results/TANIMOTO_experimental_vs_generated.png',dpi = 600)
plt.show()

In [ ]:

# Diversity 
mol_real = [Chem.MolFromSmiles(x) for x in exp_smiles_list]
fp_real = [AllChem.GetMorganFingerprintAsBitVect(y, 2) for y in mol_real]
#fp_real = [Chem.RDKFingerprint(y) for y in mol_real]

mol_gen = [Chem.MolFromSmiles(x) for x in df_generated_list if x is not None]
fp_gen = [AllChem.GetMorganFingerprintAsBitVect(y,2) for y in mol_gen]
#fp_gen = [Chem.RDKFingerprint(y) for y in mol_gen]

similarities_ij= []

for i in fp_real:
    for j in fp_gen:
        similarities_ij.append(1-DataStructs.FingerprintSimilarity(i, j))

similarities_ii= []        
for i in fp_real:
    for j in fp_real:
        similarities_ii.append(1-DataStructs.FingerprintSimilarity(i, j))
        
similarities_jj= []        
for i in fp_gen:
    for j in fp_gen:
        similarities_jj.append(1-DataStructs.FingerprintSimilarity(i, j))

similarities_ij = np.asarray(similarities_ij, dtype=np.float32)
similarities_ij = np.sort(similarities_ij)

similarities_ii = np.asarray(similarities_ii, dtype=np.float32)
similarities_ii = np.sort(similarities_ii)

similarities_jj = np.asarray(similarities_jj, dtype=np.float32)
similarities_jj = np.sort(similarities_jj)


# Plotting with shading
plt.figure(figsize=(6, 4))


p = sns.kdeplot(similarities_jj,  fill=True, #color='green',
                label='Internal diversity (Gen.) = ' + str(np.round(np.sum(similarities_jj) / (len(fp_gen)**2), 2)), linewidth=3,
                shade=True, alpha=.3)
p = sns.kdeplot(similarities_ii,  fill=True, #color='red',
                label='Internal diversity (Exp.) = ' + str(np.round(np.sum(similarities_ii) / (len(fp_real)**2), 2)), linewidth=3,
                shade=True, alpha=.3)
p = sns.kdeplot(similarities_ij,  fill=True, #color='blue',
                label='External diversity = ' + str(np.round(np.sum(similarities_ij) / (len(fp_real) * len(fp_gen)), 2)), linewidth=3,
                shade=True, alpha=.3)

p.legend(bbox_to_anchor=(0.00, 1.00), loc='upper left', fontsize=12)
#p.set(xlabel='Tanimoto coefficient', title='Distribution of Tanimoto coefficient with mean value')
#plt.tight_layout()
p.set_xlabel('Diversity', fontsize=14)  # Adjust the font size as needed
p.set_ylabel('Density', fontsize=14)  # Adjust the font size as needed
plt.tight_layout()
plt.savefig('./results/Diversity_experimental_vs_generated.png',dpi = 600)
plt.show()



In [ ]:

# Tanimoto Similarity
mol_real = [Chem.MolFromSmiles(x) for x in exp_smiles_list]
#fp_real = [AllChem.GetMorganFingerprintAsBitVect(y, 2) for y in mol_real]
fp_real = [Chem.RDKFingerprint(y) for y in mol_real]
fp_real = [MACCSkeys.GenMACCSKeys(y) for y in mol_real]

mol_gen = [Chem.MolFromSmiles(x) for x in df_generated_list if x is not None]
#fp_gen = [AllChem.GetMorganFingerprintAsBitVect(y,2) for y in mol_gen]
fp_gen = [Chem.RDKFingerprint(y) for y in mol_gen]
fp_gen = [MACCSkeys.GenMACCSKeys(y) for y in mol_gen]

similarities_ij= []

for i in fp_real:
    for j in fp_gen:
        similarities_ij.append(1-DataStructs.FingerprintSimilarity(i, j))

similarities_ii= []        
for i in fp_real:
    for j in fp_real:
        similarities_ii.append(1-DataStructs.FingerprintSimilarity(i, j))
        
similarities_jj= []        
for i in fp_gen:
    for j in fp_gen:
        similarities_jj.append(1-DataStructs.FingerprintSimilarity(i, j))


        
similarities_ij = np.asarray(similarities_ij, dtype=np.float32)
similarities_ij = np.sort(similarities_ij)

similarities_ii = np.asarray(similarities_ii, dtype=np.float32)
similarities_ii = np.sort(similarities_ii)

similarities_jj = np.asarray(similarities_jj, dtype=np.float32)
similarities_jj = np.sort(similarities_jj)


# Plotting with shading
plt.figure(figsize=(6, 4))


p = sns.kdeplot(similarities_jj,  fill=True, color='green',
                label='Mean diversity = ' + str(np.round(np.sum(similarities_jj) / (len(fp_gen)**2), 2)), linewidth=3,
                shade=True, alpha=.3)

p.legend(bbox_to_anchor=(0.00, 1.00), loc='upper left', fontsize=12)
#p.set(xlabel='Tanimoto coefficient', title='Distribution of Tanimoto coefficient with mean value')
#plt.tight_layout()
p.set_xlabel('Diversity', fontsize=14)  # Adjust the font size as needed
p.set_ylabel('Density', fontsize=14)  # Adjust the font size as needed
plt.tight_layout()
plt.savefig('./results/Diversity_experimental_vs_generated.png',dpi = 600)
plt.show()



In [ ]:
# LogP

gen_logp = [Descriptors.MolLogP(mol) for mol in mol_gen]
real_logp = [Descriptors.MolLogP(mol) for mol in mol_real]

# Plotting with shading
plt.figure(figsize=(6, 4))
p = sns.kdeplot(gen_logp,  fill=True, #color='green',
                label='Generated', linewidth=3,
                shade=True, alpha=.5)
p = sns.kdeplot(real_logp, fill=True, #color='red'
                label='Experimental',  linewidth=3,
                shade=True, alpha=.5)

#p.legend(bbox_to_anchor=(1.00, 1.00), loc='best', fontsize=12)
#p.set(xlabel='LogP', title='Distribution of LogP')
p.set_xlabel('LogP', fontsize=14)  # Adjust the font size as needed
p.set_ylabel('Density', fontsize=14)  # Adjust the font size as needed
plt.tight_layout()
plt.savefig('./results/LOGP_experimental_vs_generated.png',dpi = 600)
plt.show()

In [ ]:
# QED

gen_qed = [Descriptors.qed(mol) for mol in mol_gen]
real_qed = [Descriptors.qed(mol) for mol in mol_real]

# Plotting with shading
plt.figure(figsize=(6, 4))

p = sns.kdeplot(gen_qed,  fill=True, #color='green',
                label='Generated',  linewidth=3,
                shade=True, alpha=.5)
p = sns.kdeplot(real_qed,  fill=True, #color='red',
                label='Experimental', linewidth=3,
                shade=True, alpha=.5)

#p.legend(bbox_to_anchor=(1.00, 1.00), loc='best')
#p.set(xlabel='QED', title='Distribution of QED')
p.set_xlabel('QED', fontsize=14)  # Adjust the font size as needed
p.set_ylabel('Density', fontsize=14)  # Adjust the font size as needed
plt.tight_layout()
plt.savefig('./results/QED_experimental_vs_generated.png',dpi = 600)
plt.show()

In [ ]:
# np

fscore = npscorer.readNPModel()

gen_npscore = [npscorer.scoreMol(mol, fscore) for mol in mol_gen]
real_npscore = [npscorer.scoreMol(mol, fscore) for mol in mol_real]

# Plotting with shading
plt.figure(figsize=(6, 4))

p = sns.kdeplot(gen_npscore, fill=True, # color='green',
                label='Generated', linewidth=3,
                shade=True, alpha=.5)

p = sns.kdeplot(real_npscore,  fill=True, #color='red',
                label='Experimental', linewidth=3,
                shade=True, alpha=.5)

#p.legend(bbox_to_anchor=(1.00, 1.00), loc='best', fontsize=12)
p.set(xlabel='NP−likeness score') #, title='Distribution of NP−likeness'
p.set_xlabel('NP−likeness score', fontsize=14)  # Adjust the font size as needed
p.set_ylabel('Density', fontsize=14)  # Adjust the font size as needed
plt.tight_layout()
plt.savefig('./results/NP_experimental_vs_generated.png',dpi = 600)
plt.show()

In [ ]:
# PSA
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
import rdkit.Chem as Chem 
from rdkit.Chem import Descriptors3D
import numpy as np
import matplotlib.pyplot as plt
from rdkit.Chem.Draw import SimilarityMaps
import rdkit.Chem as Chem
from rdkit.Chem import MolSurf

gen_psa = [MolSurf.pyLabuteASA(mol) for mol in mol_gen]
real_psa = [MolSurf.pyLabuteASA(mol) for mol in mol_real]

#MolSurf.pyLabuteASA
#MolSurf.TPSA

# Plotting with shading
plt.figure(figsize=(6, 4))
p = sns.kdeplot(gen_psa,  fill=True, #color='green',
                label='Generated', linewidth=3,
                shade=True, alpha=.5)
p = sns.kdeplot(real_psa, fill=True, #color='red'
                label='Experimental',  linewidth=3,
                shade=True, alpha=.5)

#p.legend(bbox_to_anchor=(1.00, 1.00), loc='best', fontsize=12)
#p.set(xlabel='LogP', title='Distribution of LogP')
p.set_xlabel('PSA', fontsize=14)  # Adjust the font size as needed
p.set_ylabel('Density', fontsize=14)  # Adjust the font size as needed
plt.tight_layout()
plt.savefig('./results/PSA_experimental_vs_generated.png',dpi = 600)
plt.show()

In [ ]:
#HBA
gen_HBA = [rdMolDescriptors.CalcNumLipinskiHBA(mol) for mol in mol_gen]
real_HBA = [rdMolDescriptors.CalcNumLipinskiHBA(mol) for mol in mol_real]

# Plotting with shading
plt.figure(figsize=(6, 4))
p = sns.kdeplot(gen_HBA,  fill=True, #color='green',
                label='Generated', linewidth=3,
                shade=True, alpha=.5)
p = sns.kdeplot(real_HBA, fill=True, #color='red'
                label='Experimental',  linewidth=3,
                shade=True, alpha=.5)

#p.legend(bbox_to_anchor=(1.00, 1.00), loc='best', fontsize=12)
#p.set(xlabel='LogP', title='Distribution of LogP')
p.set_xlabel('HBA', fontsize=14)  # Adjust the font size as needed
p.set_ylabel('Density', fontsize=14)  # Adjust the font size as needed
plt.tight_layout()
plt.savefig('./results/HBA_experimental_vs_generated.png',dpi = 600)
plt.show()

In [ ]:
#HBD
gen_HBD = [rdMolDescriptors.CalcNumLipinskiHBD(mol) for mol in mol_gen]
real_HBD = [rdMolDescriptors.CalcNumLipinskiHBD(mol) for mol in mol_real]

# Plotting with shading
plt.figure(figsize=(6, 4))
p = sns.kdeplot(gen_HBD,  fill=True, #color='green',
                label='Generated', linewidth=3,
                shade=True, alpha=.5)
p = sns.kdeplot(real_HBD, fill=True, #color='red'
                label='Experimental',  linewidth=3,
                shade=True, alpha=.5)

p.legend(bbox_to_anchor=(1.00, 1.00), loc='best', fontsize=12)
#p.set(xlabel='LogP', title='Distribution of LogP')
p.set_xlabel('HBD', fontsize=14)  # Adjust the font size as needed
p.set_ylabel('Density', fontsize=14)  # Adjust the font size as needed
plt.tight_layout()
plt.savefig('./results/HBD_experimental_vs_generated.png',dpi = 600)
plt.show()

In [ ]:
def calculate_molecular_volume(mol):
    
    # Generate 3D coordinates
    AllChem.EmbedMolecule(mol)
    
    # Calculate volume using GetConformer and GetAtomPosition
    conformer = mol.GetConformer()
    volume = AllChem.ComputeMolVolume(mol, confId=conformer.GetId())
    
    return volume

gen_vol = [calculate_molecular_volume(mol) for mol in mol_gen]
real_vol = [calculate_molecular_volume(mol) for mol in mol_real]

# Plotting with shading
plt.figure(figsize=(6, 4))
p = sns.kdeplot(gen_vol,  fill=True, #color='green',
                label='Generated', linewidth=3,
                shade=True, alpha=.5)
p = sns.kdeplot(real_vol, fill=True, #color='red'
                label='Experimental',  linewidth=3,
                shade=True, alpha=.5)

#p.legend(bbox_to_anchor=(1.00, 1.00), loc='best', fontsize=12)
#p.set(xlabel='LogP', title='Distribution of LogP')
p.set_xlabel('Molecular volume', fontsize=14)  # Adjust the font size as needed
p.set_ylabel('Density', fontsize=14)  # Adjust the font size as needed
plt.tight_layout()
plt.savefig('./results/vol_experimental_vs_generated.png',dpi = 600)
plt.show()

# tMAP plot

In [ ]:
import pandas as pd
import tmap
from faerun import Faerun
from mhfp.encoder import MHFPEncoder
from rdkit.Chem import AllChem

In [ ]:
exp_smiles = pd.read_csv('./data/Experimental_Ligands.csv')
print('Dataset:', exp_smiles.shape)
exp_smiles

In [ ]:
exp_smiles['smiles'] = exp_smiles['smiles'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)) if Chem.MolFromSmiles(x) is not None else None)

In [ ]:
exp_smiles

In [ ]:
exp_smiles['label'] = 'real'
exp_smiles

In [ ]:
df_generated = pd.read_csv('./results/generative_model/Generated_Practical_Set.csv')
df_generated = df_generated[['smiles']]
df_generated

In [ ]:
# Assuming your DataFrame is named df
df_generated['smiles'] = df_generated['smiles'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)) if Chem.MolFromSmiles(x) is not None else None)

In [ ]:
df_generated

In [ ]:
df_generated['label'] = 'generated'
df_generated

In [ ]:
mixed_df = pd.concat([exp_smiles,df_generated], ignore_index=True)
mixed_df

In [ ]:
# Set a fixed random seed (choose any integer value)
import numpy as np
np.random.seed(42)  # Example seed value (you can change this)

In [ ]:
from collections import Counter
from matplotlib.colors import ListedColormap
from matplotlib import pyplot as plt

In [ ]:
type_labels, type_data = Faerun.create_categories(mixed_df["label"])

In [ ]:
type_labels

In [ ]:
# The number of permutations used by the MinHashing algorithm
perm = 512

# Initializing the MHFP encoder with 512 permutations
enc = MHFPEncoder(perm)

# Create MHFP fingerprints from SMILES
# The fingerprint vectors have to be of the tm.VectorUint data type
fingerprints = [tmap.VectorUint(enc.encode(s)) for s in mixed_df["smiles"]]


#df['yield'] = df['yield'].astype(str)
# Initialize the LSH Forest
lf = tmap.LSHForest(perm)

# Add the Fingerprints to the LSH Forest and index
lf.batch_add(fingerprints)
lf.index()

# Get the coordinates
x, y, s, t, _ = tmap.layout_from_lsh_forest(lf)


# Create a Faerun plot for the ESOL dataset
faerun = Faerun(view="front", coords=False, clear_color='#FFFFFF') #,clear_color='#FFFFFF'

#Hex code for white: #FFFFFF
#Hex code for black: #000000
#Hex code for red: #FF0000
#Hex code for green: #00FF00
#Hex code for blue: #0000FF


# Add scatter points with colors based on "label" values
faerun.add_scatter(
    "tmap_real_vs_generated_512",
    {
        "x": x,
        "y": y,
        "c": type_data,  # Use df['label'] directly
        "labels": mixed_df["smiles"]
    },
    point_scale=15,
    colormap=['tab10'],
    has_legend=True,
    legend_title=[''],
    legend_labels=[type_labels],
    categorical=[True],
    shader='sphere' #circle, sphere
)

# Add a tree structure connecting related points
faerun.add_tree("tmap_real_vs_generated_512_tree", {"from": s, "to": t}, point_helper="tmap_real_vs_generated_512")

# Choose the "smiles" template to display molecular structure on hover
faerun.plot('tmap_real_vs_generated_512', template="smiles", notebook_height=900)